## Task 1-3 : Climate Region Delineation

La définition des régions climatiques est un exercice qui vise à définir des régions hétérogènes dans le secteur à l'étude basé sur ses caractéristiques climatiques et physiographiques. Les données de ERA5-Land sont utlisées afin d'évaluer un ensemble de variables explicatives spatialisées et permettront d'évaluer un regroupement (<em>clustering</em>) de régions aux moyens d'algorithmes de partitionnement.

In [ ]:
import xarray as xr
import fsspec
from distributed import Client
import hvplot.xarray
import numpy as np

## Intrants

Définition d'une région spatiale et d'une période climatique afin de limiter l'extraction des grilles météos

In [ ]:
latlngbox = [-82, -74, 44.5, 49]
climatic_period = ['1981-01-01', '2010-12-31']

Définition des caracrétistiques du stockage des données ERA5-Land

In [ ]:
bucket = 's3://era5-atlantic-northeast/zarr/land/reanalysis'
storage_options = {'endpoint_url': 'https://s3.us-east-2.wasabisys.com',
                   'anon': True}

## Client Dask 

Le client Dask permet de paralléliser l'ensemble du code ci-après et d'effectuer le calcul par <em>lazy loading</em>. Lorsqu'un calcul est lancé, celui-ci peut être suivi en temps réel via le tableau de bord (<em>dashboard</em>) dont l'adresse est affichée ci-dessous.

In [ ]:
client = Client()
client

## Importation de ERA5-Land 

ERA5-Land est importé vers un dataset (ds) qui facilite l'appel à des matrices multidimensionnelles (latitude, longitude, variable, temps, etc.)

In [ ]:
ds = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options),
                  consolidated=True)

Le dataset peut être virtuellement réduit avant même de lancer le calcul d'extraction des données. Ceci est possible grâce au <em>lazy loading</em>

In [ ]:
# Accès à un sous ensemble des données via le lazy loading

ds = ds.sel(time=slice(climatic_period[0], climatic_period[1]),
            longitude=slice(latlngbox[0], latlngbox[1]),
            latitude=slice(latlngbox[3], latlngbox[2]))

In [ ]:
ds

## Calcul des grilles explicatives

### Précipitation moyenne annuelle (MAP)

La variable tp comprend le champ <em>total precipitation</em>. Le champ est passé du Dataset (ds) vers un DataArray, dans ce cas-ci da_tp, lequel ne contient que l'information de la variable tp. Il convient de rappeler que jusqu'ici, aucun calcul n'a encore été entamé en raison du <em>lazy loading</em>.


In [ ]:
da_tp = ds.tp

Les données de précipitation ERA5-Land sont cumulatives sur un jour. La fonction ci-dessous permet de décumuler les grilles.

In [ ]:
da_tp_decumulated = xr.where(da_tp.time.dt.hour == 1,
                             da_tp,
                             xr.concat([da_tp.isel(time=0), da_tp.diff(dim='time')],
                                       dim='time'))

Transformation des données vers une somme annuelle des précipitations, puis moyenne temporelle sur l'ensemble des années préalablement filtrées. Notons qu'en raison du <em>lazy loading</em>, le calcul ne débute que lorsque la méthode .compute() est appelée

In [ ]:
da_tp_moyen = da_tp_decumulated\
                    .resample(time='1Y')\
                    .sum()\
                    .mean('time')\
                    .compute()

Affichage de la précipitation moyenne annuelle (1981-2010)

In [ ]:
da_tp_moyen.hvplot()

### Température moyenne saisonnière

Conversion des unités Kelvin vers Celcius et passage des données de température moyenne à 2 mètres (t2m) du Dataset (ds) vers le DataArray da_t2m

In [ ]:
da_t2m = ds.t2m - 273.15

Définition d'un DataArray pour chacune des quatre saison de l'année. L'argument <em>test_element</em> prend en entrée une liste des mois de l'année entre 1 (janvier) et 12 (décembre)

In [ ]:
da_t2m_pr = da_t2m.sel(time = np.isin(da_t2m['time.month'], test_elements=[3,4,5,6]))

da_t2m_ete = da_t2m.sel(time = np.isin(da_t2m['time.month'], test_elements=[7,8]))

da_t2m_automne = da_t2m.sel(time = np.isin(da_t2m['time.month'], test_elements=[9,10,11]))

da_t2m_hiver = da_t2m.sel(time = np.isin(da_t2m['time.month'], test_elements=[12,1,2]))

Pour chacune des saisons de l'année, on opère une transformation des données vers une moyenne saisonnière des températures à 2 mètres, suivi d'une moyenne temporelle sur l'ensemble des années préalablement filtrées. Notons qu'en raison du <em>lazy loading</em>, le calcul ne débute que lorsque la méthode .compute() est appelée

In [ ]:
da_t2m_pr_moyen = da_t2m_pr\
                        .resample(time='1Y')\
                        .mean()\
                        .mean('time')\
                        .compute()

da_t2m_ete_moyen  = da_t2m_ete\
                        .resample(time='1Y')\
                        .mean()\
                        .mean('time')\
                        .compute()

da_t2m_automne_moyen  = da_t2m_automne\
                        .resample(time='1Y')\
                        .mean()\
                        .mean('time')\
                        .compute()

da_t2m_hiver_moyen  = da_t2m_hiver\
                        .resample(time='1Y')\
                        .mean()\
                        .mean('time')\
                        .compute()

Affichage dynamique des grilles saisonnières de température à 2 mètres pour chacune des saisons

In [ ]:
(da_t2m_pr_moyen.hvplot(label='Printemps', cmap='bwr') + \
 
 da_t2m_ete_moyen.hvplot(label='Été', cmap='bwr') + \
 
 da_t2m_automne_moyen.hvplot(label='Automne', cmap='bwr') + \
 
 da_t2m_hiver_moyen.hvplot(label='Hiver', cmap='bwr')).cols(1)